In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def gfuncsq_nondeg_sigma(i, j, sigma, eigvecs, nmin, nmax):
    elem = 0
    for alpha in range(2):
        for alpha_p in range(2):
            for beta in range(2):
                for beta_p in range(2):
                    for gamma in range(0, 2):
                        for gamma_p in range(0, 2):
                            for n in range(nmin, nmax):
                                term1 = np.conj(sigma[alpha_p, gamma_p]) * sigma[alpha, gamma]
                                term2 = np.conj(eigvecs[2*i+gamma_p, n]) * eigvecs[2*i+gamma, n]
                                term3 = np.conj(eigvecs[2*j+beta, n]) * eigvecs[2*j+beta_p, n]
                                elem += term1 * term2 * term3
    
    return elem

In [5]:
def gfuncsq_mat_nondeg_sigma(sigma, eigvecs, L, nmin, nmax):
    matrix1 = np.zeros((L*L, 2*L*L), dtype=complex)
    matrix2 = np.zeros((L*L, 2*L*L), dtype=complex)

    # Create the first matrix
    for gamma in range(2):
        for gamma_p in range(2):
            # Calculating the coefficient
            coeff = 0
            for alpha in range(2):
                for alpha_p in range(2):
                    coeff += np.conj(sigma[alpha_p,gamma]) * sigma[alpha, gamma_p]

            # Calculate the C_γγ' matrix
            C_g_gp = np.empty((L*L, 2*L*L), dtype=complex)
            for i in range(L*L):
                for n in range(2*L*L):
                    C_g_gp[i, n] = np.conj(eigvecs[2*i+gamma,n]) * eigvecs[2*i+gamma_p,n]
            
            matrix1 += coeff * C_g_gp
            matrix2 += C_g_gp

    return matrix1[:,nmin:nmax] @ matrix2[:,nmin:nmax].conj().T

In [6]:
L = 4

# Create fake eigenvectors
eigvecs = np.random.normal(loc=0, scale=1, size=(2*L*L, 2*L*L)) \
    + 1j*np.random.normal(loc=0, scale=1, size=(2*L*L, 2*L*L))

# Normalize each eigenvector
for i in range(0, 2*L*L):
    eigvecs[:, i] /= np.sqrt(np.sum(np.abs(eigvecs[:, i])**2))

sigma = np.array([[1.1, 0.2], [-0.1, 0.6]])

nmin = 4
nmax = 9

# %timeit populate(L)
original_gfuncsq = np.empty((L*L,L*L), dtype=complex)

for i in range(L*L):
    for j in range(L*L):
        original_gfuncsq[i, j] = gfuncsq_nondeg_sigma(i, j, sigma, eigvecs, nmin, nmax)

opti_gfuncsq = gfuncsq_mat_nondeg_sigma(sigma, eigvecs, L, nmin, nmax)

diff = opti_gfuncsq - original_gfuncsq
print(np.sum(np.abs(diff)))
print(np.max(np.abs(diff)))

2.556009454447473e-15
1.116197400121705e-16
